<a href="https://colab.research.google.com/github/pcsilcan/aed/blob/master/20202/aed_20202_131_avl_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AVL Balanced Tree

In [ ]:
%%writefile avl.h
#ifndef __AVL_H__
#define __AVL_H__

#include <functional>

using std::function;

template <typename T, typename R=T, T NONE=0>
class Avl {
    struct Node;

    typedef function<R(T)>      lbdKey;
    typedef function<void(T)>   lbdProc;

    Node*   root;
    int     len;
    lbdKey  key;

public:
    Avl(lbdKey key=[](T a) {return a;}) : key(key), root(nullptr), len(0) {}
    ~Avl() { destroy(root); }

    int     height()                { return height(root); }
    int     size()                  { return len; }
    int     clear()                 { destroy(root); len = 0; }

    void    add(T elem)             { add(root, elem); }
    void    inOrder(lbdProc proc)   { inOrder(root, proc); }

    void    remove(R attr);
    T       find(R attr);

private:
    void    destroy(Node*& node);
    int     height(Node* node);
    void    add(Node*& node, T elem);
    void    remove(Node*& node, R attr);
    void    inOrder(Node* node, lbdProc proc);

    Node*&  greater(Node*& node);
    Node*&  find(Node*& node, R attr);

    void    updateHeight(Node* node);
    void    rotateLeft(Node*& node);
    void    rotateRight(Node*& node);
    void    balance(Node*& node);

    Node* dummynull = nullptr;

};

#include "node.cpp"
#include "avl.cpp"
#include "balance.cpp"

#endif

In [ ]:
%%writefile node.cpp
#include "avl.h"


template <typename T, typename R=T, T NONE>
struct Avl<T, R, NONE>::Node {
    T       element;
    Node*   left;
    Node*   right;
    int     height;

    Node(T element): element(element),left(nullptr),right(nullptr),height(0) {}
};

In [ ]:
%%writefile avl.cpp
#include "avl.h"

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::destroy(Node*& node) {
    if (node != nullptr) {
        destroy(node->left);
        destroy(node->right);
        delete node;
        node = nullptr;
    }
}

template <typename T, typename R=T, T NONE>
int     Avl<T, R, NONE>::height(Node* node) {
    return node == nullptr? -1: node->height;
}

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::add(Node*& node, T elem) {
    if (node == nullptr) {
        node = new Node(elem);
        ++len;
    } else {
        if (key(elem) < key(node->element)) {
            add(node->left, elem);
        } else {
            add(node->right, elem);
        }
        balance(node); /* :O */
    }
}

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::remove(R attr) {
    // TODO do it yourself!
}
template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::remove(Node* node, R attr) {
    // TODO do it yourself!
}

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::inOrder(Node* node, lbdProc proc) {
    if (node != nullptr) {
        inOrder(node->left, proc);
        proc(node->element);
        inOrder(node->right, proc);
    }
}

template <typename T, typename R=T, T NONE>
Node*&  Avl<T, R, NONE>::greater(Node*& node) {
    return node->right != nullptr? node : greater(node->right);
}

template <typename T, typename R=T, T NONE>
T       Avl<T, R, NONE>::find(R attr) {
    Node*& node = find(root, attr);
    return node == nullptr? NONE : node->element;
}

template <typename T, typename R=T, T NONE>
Node*&  Avl<T, R, NONE>::find(Node*& node, R attr) {
    if (node == nullptr) {
        return dummynull;
    } else if (val == key(node->element)) {
        return node;
    } else if (val < key(node->element)) {
        return find(node->left, attr);
    } else {
        return find(node->right, attr);
    }
}


In [ ]:
%%writefile balance.cpp
#include "avl.h"

#define max(a, b) (((a) > (b))? (a) : (b))

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::updateHeight(Node* node) {
    if (node != nullptr) {
        node->height = max(height(node->left), height(node->right)) + 1;
    }
}

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::rotateLeft(Node*& node) { // X = node, Y = node->right
    Node* aux = node->right;
    node->right = aux->left;
    updateHeight(node);
    aux->left = node;
    updateHeight(aux);
    node = aux;
}

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::rotateRight(Node*& node) { // Y = node, X = node->left
    Node* aux = node->left;
    node->left = aux->right;
    updateHeight(node);
    aux->right = node;
    updateHeight(aux);
    node = aux;
}

template <typename T, typename R=T, T NONE>
void    Avl<T, R, NONE>::balance(Node*& node) {
    int hl = height(node->left);
    int hr = height(node->right);

    if (hr - hl < -1) {
        hl = height(node->left->left);
        hr = height(node->left->right);
        if (hr > hl) {
            rotateLeft(node->left);
        }
        rotateRight(node);
    } else (hr - hl > 1) {
        hl = height(node->right->left);
        hr = height(node->right->right);
        if (hl > hr) {
            rotateRight(node->right);
        }
        rotateLeft(node);
    } else {
        updateHeight(node);
    }
}
